In [ ]:
import xarray as xr
from uwnet.model import model_factory, call_with_xr
from src.sacred import get_run, get_last_model
import torch
import pprint
from ipywidgets import interact

In [ ]:
run_id = 85
run = get_run(run_id)
print(f"Run configuration for id {run_id}:")
print("------------------")
pprint.pprint(run['config'])
print("Loading model from PyMongo GridFS")
model_dict = torch.load(get_last_model(run_id))['dict']
model = model_factory().from_dict(model_dict)

In [ ]:
ds = xr.open_dataset("../data/processed/training.nc").isel(z=model.heights)

In [ ]:
@interact
def plot_column_plot(y=[4,32]):

    subset = ds.isel(y=slice(y,y+1), x=slice(0,1), time=slice(0,200))
    output = call_with_xr(model, subset, n_prog=1)

    plt.figure()
    output['FQTNN'].plot(x='time')

    plt.figure()
    subset.FQT.plot(x='time')

    plt.figure()
    (output.QT * subset.layer_mass).sum('z').plot(label='NN')
    (subset.QT * subset.layer_mass).sum('z').plot(label='OBS')

In [ ]:
plot_column_plot(32)

In [ ]:
plot_column_plot(4)